# Configuration Notes

Configuring device consists of four stages as shown in figure below, plus an initial set of reset instructions.

![hello](../doc/diagram.png)

# Config Notes

Variable $x$ refers to PLLs A or B.

Variable $y$ refers to CLKs 0 through 7.

## Input Stage

**Registers**: XTAL_CL, CLKIN_DIV
    
Input frequency: $f_{IN}$ determined by input source selected from one of CLKIN, XTAL, or VC.



In [15]:
800/25

32.0

## Synthesis Stage 1 (PLL)

**Registers**:
- $N_x$: MSNx_P1 (18 bits), MSNx_P2 (20 bits), MSNx_P3 (20 bits)
- FBx_INT


Generate PLL intermediate frequency.  Many documents refer to this signal as $f_{VCO}$, but I'm calling it $f_{PLLx}$.  Phase control is performed later in units of one fourth the period of this signal.  If I want high phase resolution, I should force this intermediate frequency as high as possible.
 
$f_{PLLx} = f_{IN} \times N_{x}$

where

$N_{x} = a + \frac{b}{c}$ and must be in the range 15 - 90.

**How**: Given $f_{IN}$ and desired PLL frequency $f_{PLLx}$, start things off by computing $N_x \approx \frac{f_{PLLx}}{f_{IN}}$.  See below for a, b, and c parameters for encoding this number.

Valid $N_{x}$ ratios are 4, 6, 8, and any fractional value between (15 + 0/1,048,575) and (90 + 0/1,048,575).

If $N_x$ is an event integer then also set FBx_INT to 1.

## Synthesis Stage 2 (MS)

Generate MultiSynth clock frequencies in the range 500 kHz and 150 MHz.

$f_{MSy} = f_{IN} \times \frac{N_{x}}{M_{y}}$

where

$M_{y} = p + \frac{q}{r}$ and must be in the range 6 - 1800.

Notes:
- MS6 and MS7 do not have parameters $q$ and $r$.
- CLK6 and CLK7 MultiSynth divide ratios are restricted to even integer in the range 6 to 254.


Registers: MSx_P1 (18 bits), MSx_P2 (20 bits), MSx_P3 (20 bits)



## Output Stage

Final divider $R_y$ is a power of 2: 1, 2, 4, 8, ..., 128.

$f_{CLKy} = f_{IN} \times \frac{N_{x}}{M_{y} \times R_{y}}$


# Multiplier Encoding

Frequency multiplier/dividers $M_{PLL}$ and $M_{MS}$ are each encoded using the following scheme:

$P_1 = 128 \times a + Floor\left(128 \times \frac{b}{c}\right) - 512$

$P_2 = 128 \times b - c \times Floor\left(128 \times \frac{b}{c}\right)$

$P_3 = c$


# Thoughts

Options:

- set $M_{PLL}$ to an integer (15 - 90) and $M_{MS}$ to an integer (6 - 1800)
- what if integers don't work for a scenario?  Am I better off selecting PLL or MS as fractional?
    - Fractional ratios introduce jitter (phase noise)
    - If I set PLL as fraction, then all downstream MS inherit that noise, but at least that noise is *common* across all downstream clocks.  That's a good thing right?
    
Maybe I should forget about fiddling with different frequencies, and instead focus on manipulating phase lags of individual clocks.

What is the minimum frequency resolution I want?  Better than 1 ppm for sure, relative to an output signal at 28.8 MHz.  That corresponds to 2.88 Hz resolution.  What if I shoot for 1 Hz?

What numbers do I get if I stick with even integers top and bottom?

In [76]:
f_src = 25e6  # MHz
f_targ = 28.8e6

ratio = f_targ / f_src

In [88]:
N = 91
M = 79
R = 1

In [89]:
f_PLL = f_src * N

In [90]:
f_CLK = f_src * N / M / R

In [91]:
f_CLK 

28797468.35443038

In [92]:
M = fractions.Fraction(ratio)

M = M.limit_denominator(60)

M

Fraction(53, 46)

In [93]:
f_est = f_src * M

f_est

28804347.826086957

In [94]:
f_est - f_targ

4347.826086957008

In [78]:
MS_max = 1800
M.limit_denominator(1800)

Fraction(144, 125)

In [42]:
O=1/M

In [43]:
O.limit_denominator(90)

Fraction(91, 79)

In [ ]:
r_est = 

In [ ]:
f_est = f_src * r_est

In [32]:
L = M.limit_denominator(90)

L

Fraction(91, 79)

In [34]:
L - r

0.28384317862165953

In [23]:
float(M.limit_denominator(900))

0.8680555555555556

In [25]:
M.limit_denominator(900)

Fraction(125, 144)

---

# Define clock instance

In [5]:
import fractions

import numpy as np

from snail.Si5351_Clock import clock

In [6]:
C = clock.Clock()

# Reset device

In [7]:
# Specify clock state when its disabled
C['CLK0_DIS_STATE'] = clock.constants.CLK_DIS_STATE_LOW
C['CLK1_DIS_STATE'] = clock.constants.CLK_DIS_STATE_LOW
C['CLK2_DIS_STATE'] = clock.constants.CLK_DIS_STATE_LOW
C['CLK3_DIS_STATE'] = clock.constants.CLK_DIS_STATE_LOW
C['CLK4_DIS_STATE'] = clock.constants.CLK_DIS_STATE_LOW
C['CLK5_DIS_STATE'] = clock.constants.CLK_DIS_STATE_LOW
C['CLK6_DIS_STATE'] = clock.constants.CLK_DIS_STATE_LOW
C['CLK7_DIS_STATE'] = clock.constants.CLK_DIS_STATE_LOW

# Disable all clocks
C['CLK0_OEB'] = clock.constants.CLK_OEB_DISABLE
C['CLK1_OEB'] = clock.constants.CLK_OEB_DISABLE
C['CLK2_OEB'] = clock.constants.CLK_OEB_DISABLE
C['CLK3_OEB'] = clock.constants.CLK_OEB_DISABLE
C['CLK4_OEB'] = clock.constants.CLK_OEB_DISABLE
C['CLK5_OEB'] = clock.constants.CLK_OEB_DISABLE
C['CLK6_OEB'] = clock.constants.CLK_OEB_DISABLE
C['CLK7_OEB'] = clock.constants.CLK_OEB_DISABLE

# Power down all output drivers
C['CLK0_PDN'] = clock.constants.CLK_PDN_OFF
C['CLK1_PDN'] = clock.constants.CLK_PDN_OFF
C['CLK2_PDN'] = clock.constants.CLK_PDN_OFF
C['CLK3_PDN'] = clock.constants.CLK_PDN_OFF
C['CLK4_PDN'] = clock.constants.CLK_PDN_OFF
C['CLK5_PDN'] = clock.constants.CLK_PDN_OFF
C['CLK6_PDN'] = clock.constants.CLK_PDN_OFF
C['CLK7_PDN'] = clock.constants.CLK_PDN_OFF

# Set interrupt masks to null thus allowing all asserts to go through
C['SYS_INIT_MASK'] = 0
C['LOL_A_MASK'] = 0
C['LOL_B_MASK'] = 0
C['LOS_MASK'] = 0 

# Fanout options
C['CLKIN_FANOUT_EN'] = clock.constants.FANOUT_DISABLE
C['XO_FANOUT_EN'] =    clock.constants.FANOUT_DISABLE
C['MS_FANOUT_EN'] =    clock.constants.FANOUT_DISABLE

# Input Stage



In [8]:
# Crystal's internal load capacitance
C['XTAL_CL'] = clock.constants.XTAL_CL_10PF   # XTAL_CL_6PF or XTAL_CL_8PF or XTAL_CL_10PF (default)

# Input clock frequency divider
C['CLKIN_DIV'] = clock.constants.CLKIN_DIV_1  # CLKIN_DIV_1, CLKIN_DIV_2, CLKIN_DIV_4 or CLKIN_DIV_8

 # Synthesis Stage 1 (PLL A and B)

In [5]:
# PLL source: PLL_SRC_XTAL (Si5351 A/B/C) or PLL_SRC_CLKIN (Si5351C only)
C['PLLA_SRC'] = clock.constants.PLL_SRC_XTAL
C['PLLB_SRC'] = clock.constants.PLL_SRC_XTAL

# given f_IN and desired PLL frequency, solve for N_x.  Then encode N_x via a, b, c parameters.

# see AN619 section 3.2, p. 3.

# idea: write a single function that determines values for MSN{A,B}_P{1,2,3} plus corresponding value for FB_INT.
# set to particular x clock making the calculation.
# allow for CLKIN or XTAL as source, driven by above setting for PLL{A,B}_SRC 

# compute a, b, and c
# compute and store MSN_P{1,2,3}
# test for integers, set FB{A,B}_INT

- MSNx_P1: 
- MSNx_P2:
- MSNx_P3:

- FBx_INT:

SyntaxError: invalid syntax (<ipython-input-5-bdb56bb032fe>, line 15)

# Synthesis Stage 2 (MultiSynths 0 - 7)

In [7]:
# Select PLL A as source for each MultiSynth
C['MS0_SRC'] = clock.constants.MS_SRC_PLL_A
C['MS1_SRC'] = clock.constants.MS_SRC_PLL_A
C['MS2_SRC'] = clock.constants.MS_SRC_PLL_A
C['MS3_SRC'] = clock.constants.MS_SRC_PLL_A
C['MS4_SRC'] = clock.constants.MS_SRC_PLL_A
C['MS5_SRC'] = clock.constants.MS_SRC_PLL_A
C['MS6_SRC'] = clock.constants.MS_SRC_PLL_A
C['MS7_SRC'] = clock.constants.MS_SRC_PLL_A

# Disable divide-by-4
C['MS0_DIVBY4'] = clock.constants.MS_DIVBY4_DISABLE
C['MS1_DIVBY4'] = clock.constants.MS_DIVBY4_DISABLE
C['MS2_DIVBY4'] = clock.constants.MS_DIVBY4_DISABLE
C['MS3_DIVBY4'] = clock.constants.MS_DIVBY4_DISABLE
C['MS4_DIVBY4'] = clock.constants.MS_DIVBY4_DISABLE
C['MS5_DIVBY4'] = clock.constants.MS_DIVBY4_DISABLE
# C['MS6_DIVBY4'] = clock.constants.MS_DIVBY4_DISABLE   # does not exist
# C['MS7_DIVBY4'] = clock.constants.MS_DIVBY4_DISABLE   # does not exist


# Set various MS P1, P2, P3 values

# INT

# Output Stage (Clocks 1 - 7)

In [10]:
# CLK source: match output CLK with corresponding Stage-2 MultiSynth.  More complicated options are available.
C['CLK0_SRC'] = clock.constants.CLK_SRC_MS
C['CLK1_SRC'] = clock.constants.CLK_SRC_MS
C['CLK2_SRC'] = clock.constants.CLK_SRC_MS
C['CLK3_SRC'] = clock.constants.CLK_SRC_MS
C['CLK4_SRC'] = clock.constants.CLK_SRC_MS
C['CLK5_SRC'] = clock.constants.CLK_SRC_MS
C['CLK6_SRC'] = clock.constants.CLK_SRC_MS
C['CLK7_SRC'] = clock.constants.CLK_SRC_MS

C['CLK0_INV'] = clock.constants.CLK_INV_FALSE
C['CLK1_INV'] = clock.constants.CLK_INV_FALSE
C['CLK2_INV'] = clock.constants.CLK_INV_FALSE
C['CLK3_INV'] = clock.constants.CLK_INV_FALSE
C['CLK4_INV'] = clock.constants.CLK_INV_FALSE
C['CLK5_INV'] = clock.constants.CLK_INV_FALSE
C['CLK6_INV'] = clock.constants.CLK_INV_FALSE
C['CLK7_INV'] = clock.constants.CLK_INV_FALSE

# Clock drive current (what's the best setting here?)
C['CLK0_IDRV'] = clock.constants.CLK_IDRV_8
C['CLK1_IDRV'] = clock.constants.CLK_IDRV_8
C['CLK2_IDRV'] = clock.constants.CLK_IDRV_8
C['CLK3_IDRV'] = clock.constants.CLK_IDRV_8
C['CLK4_IDRV'] = clock.constants.CLK_IDRV_8
C['CLK5_IDRV'] = clock.constants.CLK_IDRV_8
C['CLK6_IDRV'] = clock.constants.CLK_IDRV_8
C['CLK7_IDRV'] = clock.constants.CLK_IDRV_8

# Initial phase values
C['CLK0_PHOFF'] = clock.constants.CLK_PHOFF_ZERO
C['CLK1_PHOFF'] = clock.constants.CLK_PHOFF_ZERO
C['CLK2_PHOFF'] = clock.constants.CLK_PHOFF_ZERO
C['CLK3_PHOFF'] = clock.constants.CLK_PHOFF_ZERO
C['CLK4_PHOFF'] = clock.constants.CLK_PHOFF_ZERO
C['CLK5_PHOFF'] = clock.constants.CLK_PHOFF_ZERO
# C['CLK6_PHOFF'] = clock.constants.CLK_PHOFF_ZERO  # does not exist
# C['CLK7_PHOFF'] = clock.constants.CLK_PHOFF_ZERO  # does not exist


# Clock final divide (unity)
C['R0_DIV'] = clock.constants.R_DIV_1
C['R1_DIV'] = clock.constants.R_DIV_1
C['R2_DIV'] = clock.constants.R_DIV_1
C['R3_DIV'] = clock.constants.R_DIV_1
C['R4_DIV'] = clock.constants.R_DIV_1
C['R5_DIV'] = clock.constants.R_DIV_1
C['R6_DIV'] = clock.constants.R_DIV_1
C['R7_DIV'] = clock.constants.R_DIV_1